# Goodreads Data Analysis
### Michael Barmada, mdb120@pitt.edu
---

In [1]:
import pandas as pd
import nltk

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

First thing's first, we've got to load in our data. We'll call the new data frame reviews_df.

In [2]:
reviews_df = pd.read_csv('compiled_df.csv')

In [3]:
reviews_df.head()
reviews_df.tail()

,book_title,rating,text
0,The Way of Kings,5,I have a Booktube channel now! Subscribe here:...
1,The Way of Kings,3,"A three and a half star read.""What?"" Sanderson..."
2,The Way of Kings,5,"WOW. Ok, so I actually cried during this book ..."
3,The Way of Kings,5,Watch this space for an updated 2020 review! Y...
4,The Way of Kings,5,Me? Giving a Brandon Sanderson's book 5 stars?...


,book_title,rating,text
92,Dune,2,I picked this book up at last thanks to my bud...
93,Dune,2,I think teenage me would have liked this a lot...
94,Dune,1,I know this book is HUGE in the science-fictio...
95,Dune,1,Meh! Only finished this out of sheer stubbornn...
96,Dune,2,The Best Buddy Un-Read with Silvana and Elena ...


As we can see, the data from book_scraper.py is a little unorganized. We'll have to fix that, but first let's add on to our data...

In [4]:
reviews_df['toks'] = reviews_df['text'].map(lambda t: len(nltk.word_tokenize(t))) # Adds token column for each review
reviews_df['types'] = reviews_df['text'].map(lambda t: len(set(nltk.word_tokenize(t)))) # Adds type column
reviews_df['ttr'] = reviews_df.types/reviews_df.toks # Calculates ttr for each review

In [5]:
reviews_df.head()

,book_title,rating,text,toks,types,ttr
0,The Way of Kings,5,I have a Booktube channel now! Subscribe here:...,946,393,0.415433
1,The Way of Kings,3,"A three and a half star read.""What?"" Sanderson...",597,317,0.530988
2,The Way of Kings,5,"WOW. Ok, so I actually cried during this book ...",182,108,0.593407
3,The Way of Kings,5,Watch this space for an updated 2020 review! Y...,35,32,0.914286
4,The Way of Kings,5,Me? Giving a Brandon Sanderson's book 5 stars?...,12,11,0.916667


Now that our dataframe is a little more fleshed out, let's clean it up. First, we'll give it a MultiIndex with book_title and rating as our two indices. Then, we'll sort the reviews themselves in descending order based on score.

In [6]:
reviews_df.set_index(["book_title", "rating"], inplace = True)
reviews_df.sort_values(['book_title','rating'], ascending = False, inplace = True)

In [7]:
reviews_df.sort_values(['book_title','rating'], ascending = False, inplace = True)

In [8]:
reviews_df.head(15)
reviews_df.tail(15)

text  \
book_title       rating                                                      
The Way of Kings 5       I have a Booktube channel now! Subscribe here:...   
                 5       WOW. Ok, so I actually cried during this book ...   
                 5       Watch this space for an updated 2020 review! Y...   
                 5       Me? Giving a Brandon Sanderson's book 5 stars?...   
                 5       \n\n“In the end, all men die. How you lived wi...   
                 5            My video review https://youtu.be/mDxFNJ1P_ek   
                 5       So, a buddy of mine has been trying to get me ...   
                 5       Reread 2020 Bridge 4 is everything! I even own...   
                 5       4/29/18: even better the second time. Reviewed...   
                 5       The Way Of Kings (The Stormlight Archive,#1)Ho...   
                 5       “Life before Death.Strength before Weakness.Jo...   
                 4       Actual Rating: 4.5 StarsSo finishing this nove...   
                 4       The Way of Kings (The Stormlight Archive #1), ...   
                 4       \nfull review posted\nmy eyes are bleeding, my...   
                 4       ”Life before death. Strength before weakness. ...   

                         toks  types       ttr  
book_title       rating                         
The Way of Kings 5        946    393  0.415433  
                 5        182    108  0.593407  
                 5         35     32  0.914286  
                 5         12     11  0.916667  
                 5       2577    820  0.318199  
                 5          6      6  1.000000  
                 5        429    219  0.510490  
                 5        289    131  0.453287  
                 5       1853    573  0.309228  
                 5         73     50  0.684932  
                 5        213    122  0.572770  
                 4        802    407  0.507481  
                 4       1109    651  0.587015  
                 4       1054    478  0.453510  
                 4       1233    481  0.390105

text  toks  \
book_title rating                                                            
Dune       2       I've loved science fiction my whole life, but ...   259   
           2       1st reread (also a buddy read with the World B...   366   
           2       What has mood to do with it? You fight when th...   279   
           2       Woo, I conquered the Dunes, crossed the sands ...   977   
           2       I picked this book up at last thanks to my bud...   650   
           2       I think teenage me would have liked this a lot...   416   
           2       The Best Buddy Un-Read with Silvana and Elena ...   248   
           1       If this is the gold standard against which all...  1127   
           1       I am so glad I finally fulfilled my half of th...   193   
           1       I read a few years ago that if you've not foun...   724   
           1       I'm probably going to make whoever reads this ...   325   
           1       My favourite memory of this book is when I was...   971   
           1       Terrible. To write a book, you should be able ...   278   
           1       I know this book is HUGE in the science-fictio...   283   
           1       Meh! Only finished this out of sheer stubbornn...   137   

                   types       ttr  
book_title rating                   
Dune       2         148  0.571429  
           2         211  0.576503  
           2         177  0.634409  
           2         390  0.399181  
           2         288  0.443077  
           2         219  0.526442  
           2         159  0.641129  
           1         488  0.433008  
           1         128  0.663212  
           1         306  0.422652  
           1         193  0.593846  
           1         406  0.418126  
           1         167  0.600719  
           1         152  0.537102  
           1          94  0.686131

Much better!